[2탄] 비정형 데이터는 AutoML에 때려박고(?) 시작하자! PB Score: 0.94

In [ ]:
!pip install autokeras
!pip install git+https://github.com/keras-team/keras-tuner.git@1.0.2rc4
!pip install --upgrade keras-tuner

  Cloning https://github.com/keras-team/keras-tuner.git (to revision 1.0.2rc4) to /tmp/pip-req-build-kyqh8_0j
  Running command git clone -q https://github.com/keras-team/keras-tuner.git /tmp/pip-req-build-kyqh8_0j
  Running command git checkout -q 3189ab613e022d53c511348a10aed69d821f3cd1
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2rc4-cp36-none-any.whl size=92365 sha256=571940d898109fae1426746e6771a75e7af8a016c54962d60cfbc49ae50210be
  Stored in directory: /tmp/pip-ephem-wheel-cache-1zl2syz3/wheels/f6/84/20/5b1be7aae47964b49fa6b39881440a3a23d2a73d84bc097aeb
Successfully built keras-tuner
ERROR: autokeras 1.0.12 has requirement keras-tuner>=1.0.2, but you'll have keras-tuner 1.0.2rc4 which is incompatible.
  Found existing installation: keras-tuner 1.0.2
    Uninstalling keras-tuner-1.0.2:
      Successfully uninstalled keras-tuner-1.0.2


Processing /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6/keras_tuner-1.0.2-cp36-none-any.whl
  Found existing installation: keras-tuner 1.0.2rc4
    Uninstalling keras-tuner-1.0.2rc4:
      Successfully uninstalled keras-tuner-1.0.2rc4


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import autokeras as ak

ImportError: ignored

In [ ]:
train = pd.read_csv("news_train.csv")
X = train.content.values
Y =  train['info'].values

FileNotFoundError: ignored

In [ ]:
input_node = ak.TextInput()
output_node = ak.TextToIntSequence()(input_node)
output_node = ak.Embedding()(output_node)
output_node = ak.ConvBlock(separable=True)(output_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node,
    outputs=output_node,
    overwrite=True,
    max_trials=5
    )
clf.fit(X, Y, epochs=5)
model = clf.export_model()

Trial 5 Complete [00h 06m 28s]
val_loss: 0.3494620621204376

Best val_loss So Far: 0.0020312401466071606
Total elapsed time: 00h 32m 56s
INFO:tensorflow:Oracle triggered exit
Epoch 1/5
3711/3711 [==============================] - 94s 25ms/step - loss: 0.1164 - accuracy: 0.9525
Epoch 2/5
3711/3711 [==============================] - 95s 26ms/step - loss: 0.0422 - accuracy: 0.9848
Epoch 3/5
3711/3711 [==============================] - 95s 26ms/step - loss: 0.0263 - accuracy: 0.9905
Epoch 4/5
3711/3711 [==============================] - 94s 25ms/step - loss: 0.0178 - accuracy: 0.9935
Epoch 5/5
3711/3711 [==============================] - 95s 26ms/step - loss: 0.0144 - accuracy: 0.9946
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


In [ ]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None,)]                 0         
_________________________________________________________________
expand_last_dim (ExpandLastD (None, 1)                 0         
_________________________________________________________________
text_vectorization (TextVect (None, 64)                0         
_________________________________________________________________
embedding (Embedding)        (None, 64, 128)           2560128   
_________________________________________________________________
dropout (Dropout)            (None, 64, 128)           0         
_________________________________________________________________
separable_conv1d (SeparableC (None, 62, 32)            4512      
_________________________________________________________________
separable_conv1d_1 (Separabl (None, 60, 32)           

In [ ]:
test = pd.read_csv(PATH + "/news_test.csv")
sample_submission = pd.read_csv(PATH + "/sample_submission.csv")
pred_test = model.predict(test.content.values)
sample_submission.loc[:,'info'] = np.where(pred_test> 0.5, 1,0).reshape(-1)

In [ ]:
sample_submission.loc[:,["id","info"]].to_csv("sample_submission.csv", index = False)
sample_submission

,id,info
0,NEWS00237_1,0
1,NEWS00237_2,0
2,NEWS00237_3,0
3,NEWS00237_4,0
4,NEWS00237_5,0
...,...,...
142560,NEWS09482_72,1
142561,NEWS09482_73,1
142562,NEWS09482_74,1
142563,NEWS09482_75,1
